## Data Preprocessing

In [1]:
import pandas as pd

census_track = pd.read_csv('/Users/mariajosesalgado/Desktop/DSP/2020_Census_Tracts_20241105.csv')
mta_stations = pd.read_csv('/Users/mariajosesalgado/Desktop/DSP/MTA_Subway_Stations_20241007.csv')

print(census_track.head())
print(mta_stations.head())

                                            the_geom  CTLabel  BoroCode  \
0  MULTIPOLYGON (((-74.04387761639944 40.69018767...     1.00         1   
1  MULTIPOLYGON (((-73.98449647076764 40.70950966...     2.01         1   
2  MULTIPOLYGON (((-73.99022012669926 40.71440493...     6.00         1   
3  MULTIPOLYGON (((-73.9883662631772 40.716445702...    14.01         1   
4  MULTIPOLYGON (((-73.98507342254645 40.71908329...    14.02         1   

    BoroName  CT2020  BoroCT2020  CDEligibil  \
0  Manhattan     100     1000100         NaN   
1  Manhattan     201     1000201         NaN   
2  Manhattan     600     1000600         NaN   
3  Manhattan    1401     1001401         NaN   
4  Manhattan    1402     1001402         NaN   

                                             NTAName NTA2020 CDTA2020  \
0  The Battery-Governors Island-Ellis Island-Libe...  MN0191     MN01   
1                              Chinatown-Two Bridges  MN0301     MN03   
2                              Chinatown-

In [2]:
pip install geopandas

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import geopandas as gpd

bus_stops_gdf = gpd.read_file('/Users/mariajosesalgado/Desktop/DSP/nyu_2451_60059/bus_stops_nyc_dec2019.shp')

print(bus_stops_gdf.head())

   stop_id           stop_name   stop_lat   stop_lon  GEOID         NAMELSAD  \
0   400001        4 AV/E 10 ST  40.731342 -73.990292  36061  New York County   
1   400002        4 AV/E 12 ST  40.732608 -73.989958  36061  New York County   
2   400003        4 AV/E 13 ST  40.733936 -73.989720  36061  New York County   
3   400007   PARK AV S/E 21 ST  40.738976 -73.987053  36061  New York County   
4   400026  MADISON AV/E 63 ST  40.766274 -73.969329  36061  New York County   

                        geometry  
0  POINT (986940.577 205724.334)  
1  POINT (987033.092 206185.588)  
2  POINT (987098.996 206669.428)  
3  POINT (987837.856 208505.757)  
4  POINT (992746.026 218452.523)  


In [4]:
from shapely import wkt

#geometry object 
census_track['geometry'] = census_track['the_geom'].apply(wkt.loads)

#create geo data frame 
census_track_gdf = gpd.GeoDataFrame(census_track, geometry='geometry')

census_track_gdf.set_crs(epsg=4326, inplace=True)

print(census_track_gdf.head())

                                            the_geom  CTLabel  BoroCode  \
0  MULTIPOLYGON (((-74.04387761639944 40.69018767...     1.00         1   
1  MULTIPOLYGON (((-73.98449647076764 40.70950966...     2.01         1   
2  MULTIPOLYGON (((-73.99022012669926 40.71440493...     6.00         1   
3  MULTIPOLYGON (((-73.9883662631772 40.716445702...    14.01         1   
4  MULTIPOLYGON (((-73.98507342254645 40.71908329...    14.02         1   

    BoroName  CT2020  BoroCT2020  CDEligibil  \
0  Manhattan     100     1000100         NaN   
1  Manhattan     201     1000201         NaN   
2  Manhattan     600     1000600         NaN   
3  Manhattan    1401     1001401         NaN   
4  Manhattan    1402     1001402         NaN   

                                             NTAName NTA2020 CDTA2020  \
0  The Battery-Governors Island-Ellis Island-Libe...  MN0191     MN01   
1                              Chinatown-Two Bridges  MN0301     MN03   
2                              Chinatown-

In [5]:
#CRS for both data frames
print(bus_stops_gdf.crs)
print(census_track_gdf.crs)

EPSG:2263
EPSG:4326


In [6]:
if bus_stops_gdf.crs != census_track_gdf.crs:
    bus_stops_gdf = bus_stops_gdf.to_crs(census_track_gdf.crs)

In [7]:
#spatial join
# Spatial join: matching bus stops to census tracts
bus_stops_with_tracks = gpd.sjoin(bus_stops_gdf, census_track_gdf, how='left', predicate='within')

# Check the result
print(bus_stops_with_tracks.head())


   stop_id           stop_name   stop_lat   stop_lon GEOID_left  \
0   400001        4 AV/E 10 ST  40.731342 -73.990292      36061   
1   400002        4 AV/E 12 ST  40.732608 -73.989958      36061   
2   400003        4 AV/E 13 ST  40.733936 -73.989720      36061   
3   400007   PARK AV S/E 21 ST  40.738976 -73.987053      36061   
4   400026  MADISON AV/E 63 ST  40.766274 -73.969329      36061   

          NAMELSAD                    geometry  index_right  \
0  New York County  POINT (-73.99029 40.73134)       2294.0   
1  New York County  POINT (-73.98996 40.73261)       2294.0   
2  New York County  POINT (-73.98972 40.73394)       2294.0   
3  New York County  POINT (-73.98705 40.73898)         31.0   
4  New York County  POINT (-73.96933 40.76627)         73.0   

                                            the_geom  CTLabel  ...   CT2020  \
0  MULTIPOLYGON (((-73.98718049670539 40.73329214...     42.0  ...   4200.0   
1  MULTIPOLYGON (((-73.98718049670539 40.73329214...     42.

In [8]:
#loading the child oportunity index 
child_opportunity = pd.read_csv('/Users/mariajosesalgado/Desktop/DSP/raw/child_opportunity_index.csv')
print(child_opportunity.head())


        geoid  year  in100  msaid15                  msaname15  countyfips  \
0  1001020100  2010    0.0  33860.0  Montgomery, AL Metro Area        1001   
1  1001020100  2015    0.0  33860.0  Montgomery, AL Metro Area        1001   
2  1001020200  2010    0.0  33860.0  Montgomery, AL Metro Area        1001   
3  1001020200  2015    0.0  33860.0  Montgomery, AL Metro Area        1001   
4  1001020300  2010    0.0  33860.0  Montgomery, AL Metro Area        1001   

   statefips stateusps  pop  ED_APENR  ...  HE_SUPRFND   HE_RSEI  SE_POVRATE  \
0          1        AL  424  0.349463  ...  -13.815511  5.933720    9.240363   
1          1        AL  414  0.252991  ...  -13.815511  7.015026   10.677506   
2          1        AL  497  0.350239  ...  -13.815511  6.083863   10.511090   
3          1        AL  508  0.253207  ...  -13.815511  7.306938   22.413794   
4          1        AL  878  0.349768  ...  -13.815511  6.162114   11.823454   

   SE_PUBLIC    SE_HOME     SE_OCC   SE_MHE  SE_EM

In [9]:
print("Census Track Columns:", census_track.columns)
print("Child Opportunity Columns:", child_opportunity.columns)

Census Track Columns: Index(['the_geom', 'CTLabel', 'BoroCode', 'BoroName', 'CT2020', 'BoroCT2020',
       'CDEligibil', 'NTAName', 'NTA2020', 'CDTA2020', 'CDTANAME', 'GEOID',
       'Shape_Leng', 'Shape_Area', 'geometry'],
      dtype='object')
Child Opportunity Columns: Index(['geoid', 'year', 'in100', 'msaid15', 'msaname15', 'countyfips',
       'statefips', 'stateusps', 'pop', 'ED_APENR', 'ED_ATTAIN', 'ED_COLLEGE',
       'ED_ECENROL', 'ED_HSGRAD', 'ED_MATH', 'ED_READING', 'ED_SCHPOV',
       'ED_TEACHXP', 'ED_PRXECE', 'ED_PRXHQECE', 'HE_FOOD', 'HE_GREEN',
       'HE_HEAT', 'HE_HLTHINS', 'HE_OZONE', 'HE_PM25', 'HE_VACANCY', 'HE_WALK',
       'HE_SUPRFND', 'HE_RSEI', 'SE_POVRATE', 'SE_PUBLIC', 'SE_HOME', 'SE_OCC',
       'SE_MHE', 'SE_EMPRAT', 'SE_JOBPROX', 'SE_SINGLE'],
      dtype='object')


# Spatial Analysis of MTA Stations, Bus Stops, and Census Tracts

## Step-by-Step Workflow

### 1. Data Loading and Preparation
- **Objective**: Load the datasets for census tracts, MTA subway stations, child opportunity index, and bus stops.
- **Tasks**:
  - Load each dataset into an appropriate format.
  - Convert any CSV datasets containing latitude and longitude into GeoDataFrames for spatial operations.

### 2. Column Cleaning and Data Consistency
- **Objective**: Standardize and prepare data for merging and spatial joins.
- **Tasks**:
  - Strip any whitespace from column names across all datasets.
  - Ensure that `GEOID` is consistently named and formatted as a string in both census tracts and child opportunity datasets for accurate merging.

### 3. Merging Census Tract Data with Child Opportunity Index
- **Objective**: Enrich the census tract dataset by merging it with the Child Opportunity Index data.
- **Tasks**:
  - Merge on the `GEOID` field to integrate socio-economic indicators into each census tract.
  - This step creates a dataset with both geographic and socio-economic attributes.

### 4. Converting MTA and Bus Data to GeoDataFrames
- **Objective**: Prepare transit data for spatial analysis by converting to GeoDataFrames.
- **Tasks**:
  - Convert the MTA data into a GeoDataFrame by setting the geometry to the `GTFS Longitude` and `GTFS Latitude` columns.
  - Reproject all GeoDataFrames to a common CRS, such as `EPSG:4326`, to enable accurate spatial joins across datasets.

### 5. Spatial Join for MTA and Bus Stops with Census Tracts
- **Objective**: Establish a spatial relationship between transit stops and census tracts.
- **Tasks**:
  - Perform a spatial join to map each MTA station and bus stop to its corresponding census tract.
  - This links transit stops with demographic and socio-economic data within each tract, enhancing insights into transit accessibility.

### 6. Combining and Saving the Final Dataset
- **Objective**: Create a comprehensive dataset with all relevant data.
- **Tasks**:
  - Concatenate the MTA and bus stops data with their respective census tracts and the Child Opportunity Index data.
  - Drop any geometry columns before saving to a CSV file, as CSV does not support spatial data.
  - Save the resulting dataset for further analysis and sharing.




In [30]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point

# Load the datasets
census_track = pd.read_csv('/Users/mariajosesalgado/Desktop/DSP/2020_Census_Tracts_20241105.csv')
mta_stations = pd.read_csv('/Users/mariajosesalgado/Desktop/DSP/MTA_Subway_Stations_20241007.csv')
child_opportunity = pd.read_csv('/Users/mariajosesalgado/Desktop/DSP/raw/child_opportunity_index.csv')
bus_stops_gdf = gpd.read_file('/Users/mariajosesalgado/Desktop/DSP/nyu_2451_60059/bus_stops_nyc_dec2019.shp')

# Step 1: Clean column names and ensure 'GEOID' exists in both DataFrames
census_track_gdf.columns = census_track_gdf.columns.str.strip()
child_opportunity.columns = child_opportunity.columns.str.strip()

# Rename 'geoid' to 'GEOID' in child_opportunity if needed
if 'geoid' in child_opportunity.columns:
    child_opportunity.rename(columns={'geoid': 'GEOID'}, inplace=True)

# Verify column names
print("Census Tract Columns:", census_track_gdf.columns)
print("Sample Census Tract 'GEOID' values:", census_track_gdf['GEOID'].head())
print("Child Opportunity Columns:", child_opportunity.columns)
print("Sample Child Opportunity 'GEOID' values:", child_opportunity['GEOID'].head())

# Step 2: Ensure 'GEOID' columns are of string type
census_track_gdf['GEOID'] = census_track_gdf['GEOID'].astype(str)
child_opportunity['GEOID'] = child_opportunity['GEOID'].astype(str)

# Step 3: Merge the census data with child opportunity data on 'GEOID'
try:
    merged_data = census_track_gdf.merge(child_opportunity, on='GEOID', how='left')
    print("First Merge Successful. Sample of merged data:\n", merged_data.head())
except KeyError as e:
    print(f"KeyError encountered during merging: {e}")

# Step 4: Convert MTA data to GeoDataFrame using longitude and latitude
# Ensure column names are stripped in case of any leading/trailing spaces
mta_stations.columns = mta_stations.columns.str.strip()

# Rename latitude and longitude columns for MTA data
if 'GTFS Latitude' in mta_stations.columns and 'GTFS Longitude' in mta_stations.columns:
    mta_stations_gdf = gpd.GeoDataFrame(
        mta_stations, 
        geometry=gpd.points_from_xy(mta_stations['GTFS Longitude'], mta_stations['GTFS Latitude']), 
        crs="EPSG:4326"
    )
    print("MTA stations GeoDataFrame created successfully.")
    print(mta_stations_gdf.head())
else:
    print("Error: Latitude and Longitude columns are missing or misnamed in MTA data.")

# Step 5: Spatial join - map MTA and bus stops with census tracts
# Ensure both GeoDataFrames (census tracts, MTA, and bus stops) are in the same projection
census_track_gdf = census_track_gdf.to_crs("EPSG:4326")
mta_stations_gdf = mta_stations_gdf.to_crs("EPSG:4326")
bus_stops_gdf = bus_stops_gdf.to_crs("EPSG:4326")

# Spatial join to map MTA stations and bus stops to census tracts
mta_with_tracts = gpd.sjoin(mta_stations_gdf, census_track_gdf, how="left", predicate="within")
bus_stops_with_tracts = gpd.sjoin(bus_stops_gdf, census_track_gdf, how="left", predicate="within")

# Output results
print("Spatial Join for MTA with Census Tracts Successful. Sample:\n", mta_with_tracts.head())
print("Spatial Join for Bus Stops with Census Tracts Successful. Sample:\n", bus_stops_with_tracts.head())


Census Tract Columns: Index(['the_geom', 'CTLabel', 'BoroCode', 'BoroName', 'CT2020', 'BoroCT2020',
       'CDEligibil', 'NTAName', 'NTA2020', 'CDTA2020', 'CDTANAME', 'GEOID',
       'Shape_Leng', 'Shape_Area', 'geometry'],
      dtype='object')
Sample Census Tract 'GEOID' values: 0    36061000100
1    36061000201
2    36061000600
3    36061001401
4    36061001402
Name: GEOID, dtype: object
Child Opportunity Columns: Index(['GEOID', 'year', 'in100', 'msaid15', 'msaname15', 'countyfips',
       'statefips', 'stateusps', 'pop', 'ED_APENR', 'ED_ATTAIN', 'ED_COLLEGE',
       'ED_ECENROL', 'ED_HSGRAD', 'ED_MATH', 'ED_READING', 'ED_SCHPOV',
       'ED_TEACHXP', 'ED_PRXECE', 'ED_PRXHQECE', 'HE_FOOD', 'HE_GREEN',
       'HE_HEAT', 'HE_HLTHINS', 'HE_OZONE', 'HE_PM25', 'HE_VACANCY', 'HE_WALK',
       'HE_SUPRFND', 'HE_RSEI', 'SE_POVRATE', 'SE_PUBLIC', 'SE_HOME', 'SE_OCC',
       'SE_MHE', 'SE_EMPRAT', 'SE_JOBPROX', 'SE_SINGLE'],
      dtype='object')
Sample Child Opportunity 'GEOID' values: 0   

In [37]:
# Filter Child Opportunity Index data to keep only rows for New York
ny_child_opportunity = child_opportunity[child_opportunity['stateusps'] == 'NY']

# Verify the filter
print("Filtered Child Opportunity Data (NY only):\n", ny_child_opportunity.head())
print("Number of records for NY:", len(ny_child_opportunity))

# Save the filtered data to a CSV file
ny_child_opportunity.to_csv('filtered_child_opportunity_ny.csv', index=False)



Filtered Child Opportunity Data (NY only):
              GEOID  year  in100  msaid15  \
86390  36001000100  2010    1.0  10580.0   
86391  36001000100  2015    1.0  10580.0   
86392  36001000200  2010    1.0  10580.0   
86393  36001000200  2015    1.0  10580.0   
86394  36001000300  2010    1.0  10580.0   

                                    msaname15  countyfips  statefips  \
86390  Albany-Schenectady-Troy, NY Metro Area       36001         36   
86391  Albany-Schenectady-Troy, NY Metro Area       36001         36   
86392  Albany-Schenectady-Troy, NY Metro Area       36001         36   
86393  Albany-Schenectady-Troy, NY Metro Area       36001         36   
86394  Albany-Schenectady-Troy, NY Metro Area       36001         36   

      stateusps   pop  ED_APENR  ...  HE_SUPRFND   HE_RSEI  SE_POVRATE  \
86390        NY   708  0.345629  ...  -13.815511  7.456119   27.914852   
86391        NY   511  0.445770  ...  -13.815511  6.176194   26.767942   
86392        NY  2013  0.367984  ...

In [31]:
# Confirm merged data sample
print("First Merge Sample of Census Tract and Child Opportunity Data:\n", merged_data.head())

# Step 1: Prepare MTA GeoDataFrame with appropriate latitude and longitude columns
# Confirm latitude and longitude column names for MTA stations
if 'GTFS Latitude' in mta_stations.columns and 'GTFS Longitude' in mta_stations.columns:
    mta_stations_gdf = gpd.GeoDataFrame(
        mta_stations, 
        geometry=gpd.points_from_xy(mta_stations['GTFS Longitude'], mta_stations['GTFS Latitude']), 
        crs="EPSG:4326"
    )
    print("MTA stations GeoDataFrame created successfully.")
    print(mta_stations_gdf.head())
else:
    print("Error: Latitude and Longitude columns are missing or misnamed in MTA data.")

# Step 2: Reproject all GeoDataFrames to a common CRS
census_track_gdf = census_track_gdf.to_crs("EPSG:4326")
mta_stations_gdf = mta_stations_gdf.to_crs("EPSG:4326")
bus_stops_gdf = bus_stops_gdf.to_crs("EPSG:4326")

# Step 3: Spatial join for MTA stations and bus stops with Census Tracts
# MTA stations join with census tracts
mta_with_tracts = gpd.sjoin(mta_stations_gdf, census_track_gdf, how="left", predicate="within")

# Bus stops join with census tracts
bus_stops_with_tracts = gpd.sjoin(bus_stops_gdf, census_track_gdf, how="left", predicate="within")

# Step 4: Review results
print("Spatial Join for MTA with Census Tracts Successful. Sample:\n", mta_with_tracts.head())
print("Spatial Join for Bus Stops with Census Tracts Successful. Sample:\n", bus_stops_with_tracts.head())

# Optional: Merge all results into a single dataset if desired
final_data = pd.concat([mta_with_tracts, bus_stops_with_tracts, merged_data], ignore_index=True)
print("Combined Data Sample:\n", final_data.head())


First Merge Sample of Census Tract and Child Opportunity Data:
                                             the_geom  CTLabel  BoroCode  \
0  MULTIPOLYGON (((-74.04387761639944 40.69018767...     1.00         1   
1  MULTIPOLYGON (((-74.04387761639944 40.69018767...     1.00         1   
2  MULTIPOLYGON (((-73.98449647076764 40.70950966...     2.01         1   
3  MULTIPOLYGON (((-73.98449647076764 40.70950966...     2.01         1   
4  MULTIPOLYGON (((-73.99022012669926 40.71440493...     6.00         1   

    BoroName  CT2020  BoroCT2020  CDEligibil  \
0  Manhattan     100     1000100         NaN   
1  Manhattan     100     1000100         NaN   
2  Manhattan     201     1000201         NaN   
3  Manhattan     201     1000201         NaN   
4  Manhattan     600     1000600         NaN   

                                             NTAName NTA2020 CDTA2020  ...  \
0  The Battery-Governors Island-Ellis Island-Libe...  MN0191     MN01  ...   
1  The Battery-Governors Island-Ellis Is

In [32]:
# Step 1: Drop the geometry column if needed, as CSV does not support geometry data
final_data_no_geometry = final_data.drop(columns='geometry')

# Step 2: Save the resulting DataFrame to a CSV file
final_data_no_geometry.to_csv('final_transit_census_child_opportunity.csv', index=False)

print("Final data saved to 'final_transit_census_child_opportunity.csv'.")


Final data saved to 'final_transit_census_child_opportunity.csv'.


## Analysis Insights and Interpretation

### 1. MTA and Bus Stops Spatial Join with Census Tracts
- **Objective**: By joining MTA subway station and bus stop locations with NYC census tracts, we identified which transit stops are located within each census tract. This spatial relationship allows us to analyze transit accessibility at a granular level across different neighborhoods.
- **Insight**: Understanding the distribution of transit stops within census tracts can reveal accessibility patterns. Areas with higher counts of transit stops may indicate well-connected neighborhoods, while areas with fewer stops could suggest underserved regions in terms of public transportation.

### 2. Child Opportunity Index Integration
- **Objective**: Integrating the Child Opportunity Index with the census tracts dataset enriches our analysis by adding socio-economic and environmental metrics. This index offers insights into factors such as educational access, health amenities, and neighborhood stability within each tract.
- **Insight**: Census tracts with lower opportunity scores might correlate with lower accessibility to transit options. Such relationships can support targeted policy interventions aimed at improving public transportation in neighborhoods with fewer resources or higher socio-economic challenges.

### 3. Combined Transit and Opportunity Analysis
- **Objective**: By merging the transit stop data (MTA and bus) and the Child Opportunity Index, we create a comprehensive view of both physical and social accessibility in New York City neighborhoods.
- **Insight**: Areas with limited transit stops and lower Child Opportunity scores represent potential priority zones for urban improvement projects. Conversely, neighborhoods with high transit accessibility and higher opportunity scores might be considered successful models for equitable city planning.

### 4. Data Challenges and Considerations
- **Data Consistency**: Ensuring all datasets use a common Coordinate Reference System (CRS) was essential for accurate spatial joins. This helped align all geographic data precisely.
- **Handling Nulls**: During the join process, some entries may have resulted in `NaN` values, especially if certain transit stops were not associated with any census tract. These nulls indicate stops on the boundaries or in areas without assigned tracts and should be considered when interpreting results.

### 5. Next Steps for Analysis
- **Correlation Analysis**: Further statistical analysis could determine the strength of relationships between transit access and child opportunity scores.
- **Visualization**: Mapping transit access alongside the opportunity index could visually highlight disparities and serve as a communication tool for stakeholders.

**File Location**: All merged data with transit stops and opportunity indices is saved in the `final_transit_census_child_opportunity.csv` file for easy access and further analysis.
